<a href="https://colab.research.google.com/github/Juuuuls/CPE311/blob/main/HoA_7_1_Data_Collection_and_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Excercise 1**

1. Read Each file in

2. Add a column to each dataframe, called ticker, indicating the ticket symbol it is or (Apple's is AAPL, or example). This is how you look up a stock. Each file's name is also the ticker symbol, so be sue to capitalize it.

3. .Append them together into a single dataframe.

4. Save the result in a CSV file called faan.csv

In [7]:
import pandas as pd
import os

dataframes = []

directory = '/content/'
file_names = ['aapl.csv', 'amzn.csv', 'fb.csv', 'goog.csv', 'nflx.csv']

for file_name in file_names:
    ticker = os.path.splitext(file_name)[0].upper()

    file_path = os.path.join(directory, file_name)

    df = pd.read_csv(file_path)

    df['ticker'] = ticker

    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.to_csv('/content/faan.csv', index=False)

# **Excecise 2**

1. With faang, use type conversion to chane the date column into a datetime and the volume column into integers. Then , sort by date and ticke.

2. Find the seven rows with the hihest value for volume.

3. Right now, the data is somewhere between long and wide format. Use melt() to make it completely long format. Hint: date and ticker are our ID variables (they uniquely identify each row). We need to melt the rest so that we don't have seperate coolumns for open, high, low, close, and volume.


In [10]:
combined_df['date'] = pd.to_datetime(combined_df['date'])

combined_df['volume'] = combined_df['volume'].astype(int)

combined_df.sort_values(by=['date', 'ticker'], inplace=True)

In [11]:
highest_volume_rows = combined_df.nlargest(7, 'volume')

print(highest_volume_rows)

          date      open      high       low     close     volume ticker
644 2018-07-26  174.8900  180.1300  173.7500  176.2600  169803668     FB
555 2018-03-20  167.4700  170.2000  161.9500  168.1500  129851768     FB
559 2018-03-26  160.8200  161.1000  149.0200  160.0600  126116634     FB
556 2018-03-21  164.8000  173.4000  163.3000  169.3900  106598834     FB
182 2018-09-21  219.0727  219.6482  215.6097  215.9768   96246748   AAPL
245 2018-12-21  156.1901  157.4845  148.9909  150.0862   95744384   AAPL
212 2018-11-02  207.9295  211.9978  203.8414  205.8755   91328654   AAPL


In [12]:
id_vars = ['date', 'ticker']

long_format_df = pd.melt(combined_df, id_vars=id_vars, var_name='metric', value_name='value')

print(long_format_df)

           date ticker  metric         value
0    2018-01-02   AAPL    open  1.669271e+02
1    2018-01-02   AMZN    open  1.172000e+03
2    2018-01-02     FB    open  1.776800e+02
3    2018-01-02   GOOG    open  1.048340e+03
4    2018-01-02   NFLX    open  1.961000e+02
...         ...    ...     ...           ...
6270 2018-12-31   AAPL  volume  3.500347e+07
6271 2018-12-31   AMZN  volume  6.954507e+06
6272 2018-12-31     FB  volume  2.462531e+07
6273 2018-12-31   GOOG  volume  1.493722e+06
6274 2018-12-31   NFLX  volume  1.350892e+07

[6275 rows x 4 columns]


# **Excecise 3**

1. Usin web scaping, search for the list of hospitals, their address and contact information. Save the list in a new csv file, hospitals.csv.

2. Using the generated hospitals.csv, convert the csv file into pandas dataframe. Prepare the data using the necessary preprocessing techniques.


In [31]:
import csv
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_hospitals_in_Metro_Manila"

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")

    hospitals_data = []

    hospital_entries = soup.find_all("div", class_="elementor-element")

    for entry in hospital_entries:
        name = entry.find("h3", class_="elementor-heading-title").text.strip()

        address = entry.find("div", class_="elementor-text-editor").text.strip()

        contact_info = entry.find("p", class_="elementor-contact-info").text.strip() if entry.find("p", class_="elementor-contact-info") else ""

        hospitals_data.append({"Name": name, "Address": address, "Contact Information": contact_info})

    csv_file_path = "hospitals.csv"
    with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=["Name", "Address", "Contact Information"])
        writer.writeheader()
        writer.writerows(hospitals_data)

    print(f"The list of hospitals has been scraped and saved to {csv_file_path}")
else:
    print("Failed to retrieve data from the website.")


The list of hospitals has been scraped and saved to hospitals.csv


In [ ]:
import pandas as pd

df = pd.read_csv("hospitals.csv")

print("Original DataFrame:")
print(df.head())

df['Name'] = df['Name'].str.strip()
df['Address'] = df['Address'].str.strip()
df['Contact Information'] = df['Contact Information'].str.strip()

print("\nPreprocessed DataFrame:")
print(df.head())
